In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from limeImport import getSQLTable
# 如需查看每列的数据类型，可将showSchema设为True
dataSheet = getSQLTable("summary", "data/chewing.db", showSchema=True)
dataSheet

CREATE TABLE summary (
            SessionID       TEXT PRIMARY KEY NOT NULL,
            Setup           TEXT,
            Cage            INT,
            Mouse           TEXT,
            OppositeMouse   TEXT,
            Position        TEXT,
            PositionNum     INT,
            SessionDate     INT,
            StartTime       TEXT,
            Duration        REAL,
            AnalysisLength  TEXT,
            ChewCount       INT,
            ChewTime        REAL,
            ClimbCount      INT,
            ClimbTime       REAL,
            FoodBefore      REAL,
            FoodAfter       REAL,
            Food            REAL,
            Weight          REAL
        )


,SessionID,Setup,Cage,Mouse,OppositeMouse,Position,PositionNum,SessionDate,StartTime,Duration,AnalysisLength,ChewCount,ChewTime,ClimbCount,ClimbTime,FoodBefore,FoodAfter,Food,Weight
